In [1]:
import numpy as np # library to handle data in a vectorized manner

import urllib.request
import time

from urllib.request import urlopen

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0



geopy-2.1.0          | 64 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [3]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 10.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.5MB 22.3MB/s eta 0:00:01     |█████████████████               | 2.9MB 22.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup

# Part 1


In [271]:
#requests.get(url).text will ping a website and return you HTML of the website.

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

soup = BeautifulSoup(page.text, 'lxml')
#Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document.
#soup.prettify()

In [272]:
postalCode_table = soup.find(class_='wikitable sortable')
#postalCode_table

In [233]:
soup.title
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

#### The data consists 3 columns: PostalCode, Borough, and Neighborhood

In [343]:
dt = []
col = []

for index, tr in enumerate(postalCode_table.findAll('tr')):
    sec = []
    for td in tr.findAll(['th','td']):
        sec.append(td.text.rstrip())
        if (index == 0 ):
            col = sec
        else:
            dt.append(sec)

toronto_df = pd.DataFrame(data = dt, columns = col)
toronto_df.head(20).drop_duplicates().reset_index(drop=True)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [350]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned'].drop_duplicates().reset_index(drop=True)

toronto_df.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.


In [352]:
toronto_df['Neighbourhood'] =toronto_df.groupby('Postal Code')['Neighbourhood'].transform(lambda neigh: ','.
                                                                                         join(neigh))
toronto_df.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.


In [353]:
toronto_df = toronto_df['Neighbourhood'].replace("Not assigned", toronto_df["Borough"], inplace= True)

toronto_df

ValueError: Series.replace cannot use dict-value and non-None to_replace

#### Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In [327]:
clear cell output

In this notebook, I have imported and implemented all the required libraries from what I learned from the lab and online. I have completed most of the questions except the last question:" If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.". I found there's no cell which has Borough value but a Not Assigned value in the Neighbourhood column. So a error message was being generated when i tried to run the value replacement command. 

#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [355]:
toronto_df.shape

(103, 3)

# Part 2: 

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

Here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [370]:
lat_long_dt = pd.read_csv("http://cocl.us/Geospatial_data")
lat_long_dt.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [374]:
toronto_df = toronto_df.merge(lat_long_dt)
toronto_df = toronto_df.drop_duplicates().reset_index(drop=True)
toronto_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3: 

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2. to generate maps to visualize your neighborhoods and how they cluster together.

#### In this part, I have decided to focus only with boroughs that contains the word Toronto. 

In [375]:
toronto_grouped_clustering = toronto_df[toronto_df['Borough'].str.contains( 'Toronto')]
toronto_grouped_clustering.reset_index(drop=True).merge(lat_long_dt)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [379]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


from IPython.display import HTML, display


Solving environment: done

# All requested packages already installed.



In [380]:
# create map with the langitude and latitude of Toronto found from Google
longitude =  -79.3832
latitude = 43.6532
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon,label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color='green',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
display(map_clusters)

### Please view the map via this link:
 https://jupyterlab-12.labs.cognitiveclass.ai/hub/user-redirect/lab/tree/labs/DS0701EN/Coursera_Capstone/%20Segmenting-and-Clustering-Neighborhoods-in-Toronto.ipynb